refer to this for installation https://github.com/abetlen/llama-cpp-python#installation-with-openblas--cublas--clblast--metal 

In [1]:
%pip install llama_index
%pip install llama-cpp-python
%pip install transformers
%pip install torch
%pip install tqdm


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


In [2]:
import logging
import sys

from llama_index.callbacks import CallbackManager, LlamaDebugHandler
from llama_index.llms import LlamaCPP
from llama_index.llms.llama_utils import messages_to_prompt, completion_to_prompt

logging.basicConfig(stream=sys.stdout, level=logging.INFO)  # Change INFO to DEBUG if you want more extensive logging
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

llama_debug = LlamaDebugHandler(print_trace_on_end=True)
callback_manager = CallbackManager([llama_debug])

llm = LlamaCPP(
    model_url="https://huggingface.co/TheBloke/Llama-2-13B-chat-GGUF/resolve/main/llama-2-13b-chat.Q5_K_M.gguf",
    
    # optionally, you can set the path to a pre-downloaded model instead of model_url
    model_path=None,
    
    temperature=0.0,
    max_new_tokens=1024,
    
    # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
    context_window=3900,  # note, this sets n_ctx in the model_kwargs below, so you don't need to pass it there.
    
    # kwargs to pass to __call__()
    generate_kwargs={},
    
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": 4}, # I need to play with this and see if it actually helps
    
    # transform inputs into Llama2 format
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=True,
)

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 0 | VSX = 0 | 


In [3]:
# Create an index of your documents
from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext

storage_directory = "./storage"

documents = SimpleDirectoryReader('./testdata').load_data()

service_context = ServiceContext.from_defaults(llm=llm, chunk_size=1024,
                                               embed_model="local",
                                               callback_manager=callback_manager)


index = VectorStoreIndex.from_documents(documents, service_context=service_context)
# Persist the index to disk
index.storage_context.persist(persist_dir=storage_directory)

c:\Users\User\Downloads\Github\MIMI\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


**********
Trace: index_construction
    |_node_parsing ->  0.003994 seconds
      |_chunking ->  0.003994 seconds
    |_embedding ->  0.903892 seconds
**********


In [4]:
# Now you can load the index from disk when needed, and not rebuild it each time.
from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index import StorageContext, load_index_from_storage

# transcript_directory = "transcripts/ancient-aliens-official"
storage_directory = "./storage"

service_context = ServiceContext.from_defaults(llm=llm, chunk_size=1024,
                                               embed_model="local",
                                               callback_manager=callback_manager)

storage_context = StorageContext.from_defaults(persist_dir=storage_directory)
index = load_index_from_storage(storage_context, service_context=service_context)

INFO:llama_index.indices.loading:Loading all indices.
Loading all indices.
**********
Trace: index_construction
**********


In [5]:
# Query your index!
from IPython.display import Markdown, display
from llama_index.prompts import PromptTemplate

query_engine = index.as_query_engine(service_context=service_context,
                                     similarity_top_k=3)

# response = query_engine.query("What do you think of Facebook's LLaMa?")
# print(response)

# query = "What is the Alien nuclear agenda? \n\n" \
#         "Please summarize the information into 3 detailed paragraphs, something suitable for a blog post."

query = "What do you think of Facebook's LLaMa? \n\n" \
        "Please summarize the information into 3 sentences, something suitable for a tweet."

response = query_engine.query(query)
display(Markdown(f"<b>{response}</b>"))

**********
Trace: query
    |_query ->  305.296014 seconds
      |_retrieve ->  0.027924 seconds
        |_embedding ->  0.02593 seconds
      |_synthesize ->  305.267092 seconds
        |_templating ->  0.000997 seconds
        |_llm ->  305.240215 seconds
**********


<b>  Based on the provided context, here is a summary of Facebook's LLaMa in three sentences:

Facebook has publicly released LLaMa, a state-of-the-art foundational large language model designed to help researchers advance their work in this subfield of AI. The model is trained on a large set of unlabeled data and is ideal for fine-tuning for a variety of tasks. With the release of LLaMa, Facebook aims to democratize access to large-scale language models and promote responsible AI practices.</b>

In [ ]:
%pip uninstall torch torchvision torchaudio

In [ ]:
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118